# Question 1
I will built a dataframe to present the data

In [861]:
import pandas as pd
import numpy as np
from numpy import nan 
from numpy import dot
from numpy.linalg import norm
import math
import random

In [862]:
def Convert(string):
    li = list(string.split(","))
    return li
Critiques=Convert('Lisa Rose,Gene Seymour,Michael Philips,Claudia Puig,Mick Lasalle,Jack Matthews,Toby,Anne')

In [863]:
data = {
    'Lady':[2.5,3,2.5,nan,3,3,nan,1.5],
    'Snakes':[3.5,3.5,3,3.5,4,4,4.5,nan],
    'Luck':[3,1.5,nan,3,2,nan,nan,4],
    'Superman':[3.5,5,3.5,4,3,5,4,nan],
    'Dupree':[2.5,3.5,nan,2.5,2.0,3.5,1,2],
    'Night':[3,3,4,4.5,3,3,nan,nan]
    }
critiques = pd.DataFrame(data,index=Critiques).T
critiques

,Lisa Rose,Gene Seymour,Michael Philips,Claudia Puig,Mick Lasalle,Jack Matthews,Toby,Anne
Lady,2.5,3.0,2.5,NaN,3.0,3.0,NaN,1.5
Snakes,3.5,3.5,3.0,3.5,4.0,4.0,4.5,NaN
Luck,3.0,1.5,NaN,3.0,2.0,NaN,NaN,4.0
Superman,3.5,5.0,3.5,4.0,3.0,5.0,4.0,NaN
Dupree,2.5,3.5,NaN,2.5,2.0,3.5,1.0,2.0
Night,3.0,3.0,4.0,4.5,3.0,3.0,NaN,NaN


# Question 2
(a) To compute a simple similarity score, we use the Manhattan distance or an Euclidean distance.

In [864]:
def sim_distanceManhattan(person1,person2):
    sum=0
    for val1, val2 in zip(person1,person2):
        if np.isnan(val1) or np.isnan(val2):
            pass
        else:
            sum=sum+np.abs(val1-val2)
    return sum

def sim_distanceEuclidienne(person1,person2):
    sum=0
    for val1, val2 in zip(person1,person2):
        if np.isnan(val1) or np.isnan(val2):
            pass
        else:
            sum=sum+np.abs(val1-val2)*np.abs(val1-val2)
    return np.sqrt(sum)
sim_distanceEuclidienne(critiques['Lisa Rose'],critiques['Gene Seymour'])

2.3979157616563596

(b) Anne could just look at the person who has tastes most similar to her and look for a movie he likes that Anne has not seen yet. For each distance above, build the function recommendNearestNeighbor(nouveauCritique, Critiques) returning a list of movies to re- commend to the user nouveauCritique, based on the tastes of the critic who is close (similar) to him.

In [865]:
# #fill the null values via mean of that movie
# #otherwise we can not compute the distance of two persons directly
# for idx in critiques.index:
#     critiques.loc[idx]=critiques.loc[idx].fillna(critiques.loc[idx].mean())
# critiques

In [866]:
def computeNearestNeighbor(nouveauCritique, critiques):
    distances=[]
    for critique in list(critiques.columns):
        if critique!=nouveauCritique:
            distance=sim_distanceManhattan(critiques[critique],critiques[nouveauCritique])
            distances.append((distance,critique))
    distances.sort()
    return distances
computeNearestNeighbor('Toby',critiques)

[(1.0, 'Anne'),
 (2.0, 'Michael Philips'),
 (2.5, 'Claudia Puig'),
 (2.5, 'Mick Lasalle'),
 (3.0, 'Lisa Rose'),
 (4.0, 'Jack Matthews'),
 (4.5, 'Gene Seymour')]

In [867]:
def recommendNearestNeighbor(nouveauCritique, critiques):
    recommendMovies=[]
    NearestNeighbor= computeNearestNeighbor(nouveauCritique,critiques)[0][1]
    print("{}'s nearest neighbor is {}, thus we recommend the movies: ".format(nouveauCritique,NearestNeighbor))
    for i in range(0,len(critiques[nouveauCritique])):
        if np.isnan(critiques[nouveauCritique][i]) and not np.isnan(critiques[NearestNeighbor][i]):
            recommendMovies.append((critiques.index[i],critiques[NearestNeighbor][i]))
    return recommendMovies
recommendNearestNeighbor('Toby',critiques)

Toby's nearest neighbor is Anne, thus we recommend the movies: 


[('Lady', 1.5), ('Luck', 4.0)]

(c) To solve the previous limitations, you need to score the items by producing a weighted score (global score) that ranks the critics. The procedure which determine the recommendations to suggest to Anne is describe in two steps below. For a given movie a, let us denote by C(a) the list of all the critics which gave a rating to a, and x(a) the rating given to a by the critic x.

(c)i

In [868]:
def recommendationPool(targetPerson,distanceFunction,critiques):
    dict={}
    for i in range(0,len(targetPerson)):
        if np.isnan(targetPerson[i]):
            critics=[]
            total=0
            sa=0
            for j in range(0,len(critiques.iloc[i,:])):
                if not np.isnan(critiques.iloc[i,j]):
                    critics.append(critiques.columns[j])
                    if distanceFunction=="Manhattan":
                        total=total+critiques.iloc[i,j]/(1+sim_distanceManhattan(targetPerson,critiques.iloc[:,j]))
                        sa=sa+1/(1+sim_distanceManhattan(targetPerson,critiques.iloc[:,j]))
                    else: 
                        if distanceFunction=="Euclidienne":
                            total=total+critiques.iloc[i,j]/(1+sim_distanceEuclidienne(targetPerson,critiques.iloc[:,j]))
                            sa=sa+1/(1+sim_distanceEuclidienne(targetPerson,critiques.iloc[:,j]))
                        else:
                            return print("please type in a valid distance function")
            dict[critiques.index[i]]=[["C(a)",critics],["total(a)",total],["s(a)",sa],["s'(a)",total/sa]] 
    return dict

(c)ii-i

In [869]:
def Bestrecommend(targetPerson,distanceFunction,data):
    rec_dict=recommendationPool(targetPerson,distanceFunction,data)
    rec_dict_sort={}
    for i in range(0,len(list(rec_dict.keys()))):
        rec_dict_sort[list(rec_dict.keys())[i]]=rec_dict.get(list(rec_dict.keys())[i])[-1][-1]
    return min(rec_dict_sort, key=rec_dict_sort.get)

In [870]:
Bestrecommend(targetPerson=critiques['Anne'],distanceFunction="Euclidienne",data=critiques)

'Night'

In [871]:
Bestrecommend(targetPerson=critiques['Anne'],distanceFunction="Manhattan",data=critiques)

'Night'

(c)ii-ii

In [872]:
def recommendationPoolExp(targetPerson,distanceFunction,critiques):
    dict={}
    for i in range(0,len(targetPerson)):
        if np.isnan(targetPerson[i]):
            critics=[]
            total=0
            sa=0
            for j in range(0,len(critiques.iloc[i,:])):
                if not np.isnan(critiques.iloc[i,j]):
                    critics.append(critiques.columns[j])
                    if distanceFunction=="Manhattan":
                        total=total+critiques.iloc[i,j]*math.exp(-sim_distanceManhattan(targetPerson,critiques.iloc[:,j]))
                        sa=sa+math.exp(-sim_distanceManhattan(targetPerson,critiques.iloc[:,j]))
                    else: 
                        if distanceFunction=="Euclidienne":
                            total=total+critiques.iloc[i,j]*math.exp(-sim_distanceEuclidienne(targetPerson,critiques.iloc[:,j]))
                            sa=sa+math.exp(-sim_distanceEuclidienne(targetPerson,critiques.iloc[:,j]))
                        else:
                            return print("please type in a valid distance function")
            dict[critiques.index[i]]=[["C(a)",critics],["total(a)",total],["s(a)",sa],["s'(a)",total/sa]] 
    return dict
def BestrecommendwithExp(targetPerson=critiques['Anne'],distanceFunction="Manhattan",data=critiques):
    rec_dict=recommendationPoolExp(targetPerson,distanceFunction,data)
    rec_dict_sort={}
    for i in range(0,len(list(rec_dict.keys()))):
        rec_dict_sort[list(rec_dict.keys())[i]]=rec_dict.get(list(rec_dict.keys())[i])[-1][-1]
    return min(rec_dict_sort, key=rec_dict_sort.get)

In [873]:
BestrecommendwithExp(targetPerson=critiques['Anne'],distanceFunction="Euclidienne",data=critiques)

'Snakes'

In [874]:
BestrecommendwithExp(targetPerson=critiques['Anne'],distanceFunction="Manhattan",data=critiques)

'Snakes'

(d)PearsonRecommend()

In [875]:
def pearson(person1, person2):
    if len(person1)==len(person2):
        key1_list=[]
        key2_list=[]
        for i in range(0,len(person1)):
            if not np.isnan(person1[i]) and not np.isnan(person2[i]):
                key1_list.append(person1[i])
                key2_list.append(person2[i])
        return np.corrcoef(key1_list,key2_list)[0][1]
    else:
        return("error")

In [876]:
def recommendationPoolPearson(targetPerson,critiques):
    dict={}
    for i in range(0,len(targetPerson)):
        if np.isnan(targetPerson[i]):
            critics=[]
            total=0
            sa=0
            for j in range(0,len(critiques.iloc[i,:])):
                if not np.isnan(critiques.iloc[i,j]):
                    critics.append(critiques.columns[j])
                    total=total+critiques.iloc[i,j]*pearson(targetPerson,critiques.iloc[:,j])
            dict[critiques.index[i]]=[["C(a)",critics],["total(a)",total]] 
    return dict
def PearsonRecommend(targetPerson=critiques['Anne'],data=critiques):
    rec_dict=recommendationPoolPearson(targetPerson,data)
    rec_dict_sort={}
    for i in range(0,len(list(rec_dict.keys()))):
        rec_dict_sort[list(rec_dict.keys())[i]]=rec_dict.get(list(rec_dict.keys())[i])[-1][-1]
    return min(rec_dict_sort, key=rec_dict_sort.get)

In [877]:
PearsonRecommend(targetPerson=critiques['Anne'],data=critiques)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/numpy/lib/function_base.py:2845: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/numpy/lib/function_base.py:2704: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/numpy/lib/function_base.py:2704: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)


'Snakes'

(e)CosineRecommend

In [878]:
def recommendationPoolCosine(targetPerson,critiques):
    dict={}
    for i in range(0,len(targetPerson)):
        if np.isnan(targetPerson[i]):
            critics=[]
            total=0
            sa=0
            for j in range(0,len(critiques.iloc[i,:])):
                if not np.isnan(critiques.iloc[i,j]):
                    critics.append(critiques.columns[j])
                    total=total+critiques.iloc[i,j]*(dot(targetPerson,critiques.iloc[:,j])/(norm(targetPerson)*norm(critiques.iloc[:,j])))
            dict[critiques.index[i]]=[["C(a)",critics],["total(a)",total]] 
    return dict
def CosineRecommend(targetPerson=critiques['Anne'],data=critiques):
    rec_dict=recommendationPoolCosine(targetPerson,data)
    rec_dict_sort={}
    for i in range(0,len(list(rec_dict.keys()))):
        rec_dict_sort[list(rec_dict.keys())[i]]=rec_dict.get(list(rec_dict.keys())[i])[-1][-1]
    return min(rec_dict_sort, key=rec_dict_sort.get)

In [879]:
CosineRecommend()

'Snakes'

# Question 3

In [880]:
df=pd.read_csv('data/3.csv',index_col=0)
df

,Angelica,Bill,Chan,Dan,Hailey,Jordyn,Sam,Veronica
Blue Traveler,3.5,2.0,5.0,3.0,NaN,NaN,5.0,3.0
Broken Bells,2.0,3.5,1.0,4.0,4.0,4.5,2.0,NaN
Deadmau5,NaN,4.0,1.0,4.5,1.0,4.0,NaN,NaN
Norah Jones,4.5,NaN,3.0,NaN,4.0,5.0,3.0,5.0
Phoenix,5.0,2.0,5.0,3.0,NaN,5.0,5.0,4.0
Slightly Stoopid,1.5,3.5,1.0,4.5,NaN,4.5,4.0,2.5
The Strokes,2.5,NaN,NaN,4.0,4.0,4.0,5.0,3.0
Vampire Weekend,2.0,3.0,NaN,2.0,1.0,4.0,NaN,NaN


## 3.1 recommend to Veronica

In [881]:
recommendNearestNeighbor('Veronica',df)

Veronica's nearest neighbor is Hailey, thus we recommend the movies: 


[('Broken Bells', 4.0), ('Deadmau5', 1.0), ('Vampire Weekend', 1.0)]

In [882]:
Bestrecommend(targetPerson=df['Veronica'],distanceFunction="Euclidienne",data=df)

'Vampire Weekend'

In [883]:
Bestrecommend(targetPerson=df['Veronica'],distanceFunction="Manhattan",data=df)

'Vampire Weekend'

In [884]:
BestrecommendwithExp(targetPerson=df['Veronica'],distanceFunction="Euclidienne",data=df)

'Vampire Weekend'

In [885]:
BestrecommendwithExp(targetPerson=df['Veronica'],distanceFunction="Manhattan",data=df)

'Vampire Weekend'

In [886]:
PearsonRecommend(targetPerson=df['Veronica'],data=df)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/numpy/lib/function_base.py:2853: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/numpy/lib/function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


'Broken Bells'

In [887]:
CosineRecommend(targetPerson=df['Veronica'],data=df)

'Broken Bells'

## 3.2 recommend to Hailey

In [888]:
recommendNearestNeighbor('Hailey',df)

Hailey's nearest neighbor is Veronica, thus we recommend the movies: 


[('Blue Traveler', 3.0), ('Phoenix', 4.0), ('Slightly Stoopid', 2.5)]

In [889]:
Bestrecommend(targetPerson=df['Hailey'],distanceFunction="Euclidienne",data=df)

'Slightly Stoopid'

In [890]:
Bestrecommend(targetPerson=df['Hailey'],distanceFunction="Manhattan",data=df)

'Slightly Stoopid'

In [891]:
BestrecommendwithExp(targetPerson=df['Hailey'],distanceFunction="Euclidienne",data=df)

'Slightly Stoopid'

In [892]:
BestrecommendwithExp(targetPerson=df['Hailey'],distanceFunction="Manhattan",data=df)

'Slightly Stoopid'

In [893]:
PearsonRecommend(targetPerson=df['Hailey'],data=df)

'Blue Traveler'

In [894]:
CosineRecommend(targetPerson=df['Hailey'],data=df)

'Blue Traveler'

# Question 4

In [895]:
def ElaborateMyOwnExample(m,n,p,count=0):
    while True:
        count=count+1
        print("There has been No.{} times try to generate new data".format(count))
        df=pd.DataFrame(index=range(m),columns=range(n))
        for i in range(0,m):
            for j in range(0,n):
                if random.random()>p:
                    df.iloc[i,j]=random.randint(0,50)/5
        if df.isnull().sum().sum()>=0.3*m*n and df.isnull().sum().sum()<=0.5*m*n and df.isnull().all(axis='columns').any()==False:
            break
    while True:
        for j in range(0,n):
            if df[j].isnull().sum()>0.5/m:
                res_list=[
                Bestrecommend(targetPerson=df[j],distanceFunction="Manhattan",data=df),
                BestrecommendwithExp(targetPerson=df[j],distanceFunction="Euclidienne",data=df),
                BestrecommendwithExp(targetPerson=df[j],distanceFunction="Manhattan",data=df),
                PearsonRecommend(targetPerson=df[j],data=df),
                CosineRecommend(targetPerson=df[j],data=df)
                ]
                if len(res_list) == len(set(res_list)) and len(res_list)!=0:
                    return df
        if j==n-1:
            print("****************************")
            print("Recommendations are not all different, try again.".format(count))
            ElaborateMyOwnExample(m,n,p,count)
            break

In [896]:
df=ElaborateMyOwnExample(15,20,0.5,0)
df

There has been No.1 times try to generate new data
There has been No.2 times try to generate new data
There has been No.3 times try to generate new data


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/numpy/lib/function_base.py:2845: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/numpy/lib/function_base.py:2704: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/numpy/lib/function_base.py:2704: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/numpy/lib/function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/numpy/core/_methods.py:182: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,NaN,NaN,NaN,1.4,NaN,1.0,7.4,6.6,7.8,5.2,NaN,8.6,2.0,NaN,8.6,1.8,NaN,1.0,NaN,6.8
1,NaN,NaN,5.2,0.6,4.2,4.4,NaN,2.2,NaN,NaN,1.6,2.0,7.4,NaN,NaN,NaN,1.8,NaN,5.8,1.6
2,NaN,0.0,NaN,2.0,5.0,6.8,NaN,8.4,4.6,4.8,6.6,0.6,NaN,NaN,NaN,10.0,NaN,NaN,5.4,10.0
3,6.4,7.2,NaN,NaN,8.6,NaN,NaN,2.6,1.4,2.0,NaN,NaN,0.4,5.4,3.0,4.4,NaN,1.8,NaN,9.0
4,8.0,5.6,0.4,NaN,NaN,NaN,NaN,NaN,NaN,6.6,NaN,NaN,NaN,NaN,3.8,5.4,2.2,NaN,NaN,NaN
5,NaN,0.0,NaN,7.8,6.6,9.8,7.6,7.2,6.8,0.8,NaN,0.4,NaN,NaN,9.2,NaN,NaN,8.2,9.4,NaN
6,4.0,6.2,NaN,3.6,NaN,NaN,NaN,NaN,1.4,NaN,NaN,NaN,NaN,3.8,1.8,NaN,NaN,1.2,NaN,5.8
7,0.4,2.4,NaN,NaN,3.2,7.2,NaN,7.6,NaN,NaN,2.4,9.6,7.6,7.6,2.0,7.0,7.8,NaN,4.0,NaN
8,NaN,10.0,3.8,NaN,NaN,NaN,2.0,NaN,NaN,7.0,NaN,NaN,NaN,2.6,8.4,NaN,NaN,0.6,10.0,10.0
9,NaN,NaN,NaN,NaN,2.4,8.8,0.4,2.2,0.4,1.2,4.6,NaN,9.2,NaN,1.0,3.6,NaN,1.6,0.4,2.2
